In [5]:
import pandas as pd
import numpy as np
from datetime import datetime, timedelta

In [8]:
df = pd.read_csv("../files/IoT Data.csv")

df

,DeviceId,Year,Month,num_days_with_data,num_active_days,num_inactive_days,total_active_records,total_inactive_records,avg_RSSI,avg_PanelVoltage,avg_MotorCurrent,avg_MotorVoltage,avg_MotorSpeed,avg_MotorPower
0,868328054195702,2024,1,3,2,1,106,191,30.983165,35.570020,3.940104,11.661636,937.535354,136.967100
1,869604064508855,2024,1,31,22,9,3195,16755,19.797995,41.559197,1.315669,40.339038,304.541053,NaN
2,869640056869756,2024,1,31,0,31,0,19775,18.036612,39.723700,0.000000,0.000000,0.000000,0.000000
3,869640056937025,2024,1,29,4,25,817,15986,20.379158,39.516907,0.239725,1.198476,96.305660,8.678694
4,869640056981189,2024,1,19,1,18,77,12123,14.594426,40.113545,0.021611,0.132057,10.615082,0.752475
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
87808,867556041891675,2023,12,1,1,0,22,2,26.958333,37.245750,4.391250,19.372625,3312.000000,160.597269
87809,868328054304403,2023,12,1,1,0,1,48,18.571429,41.860755,0.607898,36.127816,18378.612240,24.521106
87810,868328054243825,2023,12,1,1,0,25,25,6.200000,35.554760,1.146620,11.361880,912.480000,40.880008
87811,868328054278862,2023,12,1,1,0,3,109,13.857143,40.909964,0.068750,0.464018,37.285714,2.661157


In [ ]:
print("Data loaded successfully!")
print(f"Shape: {df.shape}")
print("\nColumn names:")
print(df.columns.tolist())
print("\nFirst few rows:")
print(df.head())

/home/pius/SC/sunculture-data/science/03-Pump-usage-vs-payment-patterns/scripts


In [ ]:
def analyze_pump_usage_from_runtime_counters(metadata_df):
    """
    Uses device runtime counters from metadata reports

    """
    daily_runtime = (
        metadata_df.groupby(["device_id", metadata_df["timestamp"].dt.date])
        .agg({"runtime_counter_minutes": ["min", "max"], "timestamp": "count"})
        .reset_index()
    )

    daily_runtime.columns = [
        "device_id",
        "date",
        "min_counter",
        "max_counter",
        "report_count",
    ]

    daily_runtime["daily_runtime_minutes"] = (
        daily_runtime["max_counter"] - daily_runtime["min_counter"]
    )

    # Summary statistics per device
    device_summary = (
        daily_runtime.groupby("device_id")
        .agg({"daily_runtime_minutes": ["sum", "mean", "std"], "date": "count"})
        .reset_index()
    )

    device_summary.columns = [
        "device_id",
        "total_runtime_minutes",
        "avg_daily_runtime",
        "std_daily_runtime",
        "days_with_data",
    ]
    device_summary["total_runtime_hours"] = device_summary["total_runtime_minutes"] / 60
    device_summary["avg_daily_runtime_hours"] = device_summary["avg_daily_runtime"] / 60

    return device_summary, daily_runtime